In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D, Dropout
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('recipes_v210k.csv')#тут нужен большой и подготовленный датасет, не было времени, использовал первый попавшийся, отредактировал более менее
pd.set_option('display.max_columns', None)

# Меняю название столбца
df.rename(columns={'ingredients;': 'ingredients'}, inplace=True)

# Удаляем ненужное и пустые строки
df['url'] = df['url'].str.replace('версия для печати"','').str.replace('Рецепты', '').str.replace('>', '').str.strip()
df.dropna(subset=['url'], inplace=True)

# тут перекидываю параметры в соответствующие столбцы
df['temp'] = df['url'].str.split(',', n = 1).str[1]

df[['name', 'ingredients']] = df['temp'].str.split(',',n = 1, expand=True)

df['name'] = df['name'].str.strip()
df['ingredients'] = df['ingredients'].str.strip()

import re
import pickle

# Извлечение ссылок из столбца 'url'
df['url'] = df['url'].str.split(',', n=1).str[0]
df['url'] = df['url'].str.strip()

df.drop('temp', axis=1, inplace=True)

#проверка
#print(df.head())
# Заполнение пропущенных значений пустыми строками
df['ingredients'] = df['ingredients'].fillna('')
def clean_ingredients(ingredient_str):
    # Remove the outer quotes if they exist
    ingredient_str = ingredient_str.strip('"')

    # Parse the dictionary
    try:
        ingredient_dict = ast.literal_eval(ingredient_str)
    except (SyntaxError, ValueError):
        return ''

    # Ensure the parsed object is a dictionary
    if not isinstance(ingredient_dict, dict):
        return ''

    # Extract keys (ingredient names)
    ingredients = list(ingredient_dict.keys())

    # Join ingredients into a single string separated by commas
    clean_ingredients_str = ', '.join(ingredients)

    return clean_ingredients_str

df['ingredients'] = df['ingredients'].apply(clean_ingredients)

print(df.head())
# Удаление строк с пустыми или отсутствующими URL
df = df[df['url'].notna() & (df['url'].str.strip() != '')]

# Токенизация ингредиентов
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['ingredients'])

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Преобразование текста в последовательности
ingredients_seq = tokenizer.texts_to_sequences(df['ingredients'])

# Выравнивание последовательностей
max_len_ingredients = max([len(seq) for seq in ingredients_seq])
ingredients_seq = pad_sequences(ingredients_seq, maxlen=max_len_ingredients, padding='post')

# Кодирование URL-адресов
label_encoder = LabelEncoder()
url_encoded = label_encoder.fit_transform(df['url'])

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(ingredients_seq, url_encoded, test_size=0.2, random_state=42)

# Архитектура модели
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128  # Увеличение размерности эмбеддингов для лучшего представления
lstm_units = 256  # Увеличение количества LSTM-юнитов для лучшего обучения

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len_ingredients))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(lstm_units))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


                                             url  \
0  https://www.povarenok.ru/recipes/show/164365/   
1  https://www.povarenok.ru/recipes/show/164365/   
2  https://www.povarenok.ru/recipes/show/164365/   
3  https://www.povarenok.ru/recipes/show/164365/   
4  https://www.povarenok.ru/recipes/show/164365/   

                                 name              ingredients  
0  Густой молочно-клубничный коктейль  Молоко, Клубника, Сахар  
1  Густой молочно-клубничный коктейль  Молоко, Клубника, Сахар  
2  Густой молочно-клубничный коктейль  Молоко, Клубника, Сахар  
3  Густой молочно-клубничный коктейль  Молоко, Клубника, Сахар  
4  Густой молочно-клубничный коктейль  Молоко, Клубника, Сахар  


In [ ]:
# Обучение модели
history = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.2)  # Увеличение эпох и размера пакета

# Функция для предсказания URL-адреса по ингредиентам
def predict_recipe_url(ingredients):
    ingredients_seq = tokenizer.texts_to_sequences([ingredients])
    ingredients_seq = pad_sequences(ingredients_seq, maxlen=max_len_ingredients, padding='post')
    prediction = model.predict(ingredients_seq)
    predicted_url = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_url[0]

# Тестирование модели
test_ingredients = "Сыр твердый,Чеснок,Яйцо куриное, Грейпфрут"
predicted_url = predict_recipe_url(test_ingredients)
print('Ингредиенты:', test_ingredients)
print('Предсказанный URL-адрес рецепта:', predicted_url)


Epoch 1/20
52/52 [==============================] - 50s 863ms/step - loss: 3.8091 - accuracy: 0.0703 - val_loss: 2.7257 - val_accuracy: 0.2111
Epoch 2/20
52/52 [==============================] - 41s 783ms/step - loss: 2.5137 - accuracy: 0.2667 - val_loss: 1.6716 - val_accuracy: 0.5134
Epoch 3/20
52/52 [==============================] - 41s 800ms/step - loss: 1.7422 - accuracy: 0.4811 - val_loss: 1.0808 - val_accuracy: 0.7427
Epoch 4/20
52/52 [==============================] - 42s 803ms/step - loss: 1.2318 - accuracy: 0.6383 - val_loss: 0.9673 - val_accuracy: 0.6813
Epoch 5/20
52/52 [==============================] - 41s 791ms/step - loss: 0.8688 - accuracy: 0.7548 - val_loss: 0.5705 - val_accuracy: 0.7914
Epoch 6/20
52/52 [==============================] - 41s 794ms/step - loss: 0.7514 - accuracy: 0.7764 - val_loss: 0.4791 - val_accuracy: 0.8394
Epoch 7/20
52/52 [==============================] - 41s 798ms/step - loss: 0.5733 - accuracy: 0.8251 - val_loss: 0.4237 - val_accuracy: 0.8637

In [ ]:
#model.save('model.h5')
model.save('model.keras')
with open('max_len_ingredients.pkl', 'wb') as f:
    pickle.dump(max_len_ingredients, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

# Тестирование модели
test_ingredients = "Молоко, Сахар"
predicted_url = predict_recipe_url(test_ingredients)
print('Ингредиенты:', test_ingredients)
print('Предсказанный URL-адрес рецепта:', predicted_url)


1/1 [==============================] - 0s 38ms/step
Ингредиенты: Молоко, Сахар
Предсказанный URL-адрес рецепта: https://www.povarenok.ru/recipes/show/54737/
